# ФИНАЛЬНОЕ ЗАДАНИЕ

# Подход 1: градиентный бустинг "в лоб"

Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.


1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).


2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.


3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.


4. Какой столбец содержит целевую переменную? Запишите его название.


5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import time
import datetime

In [3]:
import pandas as pd

data_train = pd.read_csv("features_train.csv")

X_train = data_train.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', \
       'barracks_status_radiant', 'barracks_status_dire'], axis = 1)

In [22]:
X_train.head()

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,3,1430263531,1,29,4,1779,1056,14,0,0,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,4,1430282290,7,13,4,1431,1090,8,1,0,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [23]:
for k in range(X_train.shape[1]):
    if X_train.count()[k] != X_train.shape[0]:
        print(X_train.columns[k])

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение


first_blood_time: игровое время первой крови


first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)


first_blood_player1: игрок, причастный к событию


first_blood_player2: второй игрок, причастный к событию

dire\radiant_bottle_time: предмет "bottle" не был приобретен этой командой
    
    
dire\radiant_courier_time: предмет "courier" не был приобретен этой командой
    
    
dire\radiant_flying_courier_time: предмет "flying_courier" не был приобретен этой командой
    

dire\radiant_first_ward_time: предмет "ward", не был приобретен этой командой

In [24]:
X_train = X_train.fillna(0)

Целевую переменную содержит столбец "radiant_win"

In [10]:
y_train = data_train["radiant_win"]

In [26]:
from sklearn.model_selection import KFold, GridSearchCV

fk = KFold(n_splits = 5, shuffle = True)

from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()

params = {'n_estimators': [10 , 20 , 30]}

start_time = datetime.datetime.now()
gr = GridSearchCV(gbc, params, scoring = 'roc_auc', cv = fk)
gr.fit(X_train, y_train)

print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:02:09.304334


При 30-ти деревьях кросс-валидация проводилась 0:01:05.379830 сек

In [27]:
gr.best_score_

0.6896683573246216

Получилось примерно 0.6895

Имеет смысл взять больше 30-ти деревьев, потому что была достигнута верхняя граница настраиваемого параметра "n_estimators"б однако надо не забывать соотносить прирост качества и требуемых ресурсов. Чтобы ускорить обучение при увеличении количества деревьев можно органичить их максимальную глубину, увеличить параметр learning_rate, попобовать понастраивать другие параметры, уменьшить размеры выборки для обучения и кросс-валидации

# Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.


Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.


1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?


2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?


3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).


4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.


5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?


6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [4]:
X_train = data_train.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', \
       'barracks_status_radiant', 'barracks_status_dire'], axis = 1)

X_train = X_train.fillna(0)

In [19]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)

In [30]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

params_log = {'C': [0.01]}

start_time = datetime.datetime.now()
gr = GridSearchCV(lr, params_log, scoring = 'roc_auc', cv = fk)
gr.fit(X_train, y_train)

print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:00:15.407136


In [31]:
print(gr.best_params_)
print(gr.best_score_)

{'C': 0.01}
0.7163949027842432


Лучшее качество примерно 0.7166 при С = 0.01.Это выше, чем у градиентного бустинга. Возможно в данных имеются линейные зависимости или близкие к линейным, поэтому на логистической регрессии оценка качества выше. Работает быстрее чем градиентный бустинг примерно в 1,6 раз.

In [43]:
X_train = X_train.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', \
                      'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', \
                      'd5_hero'], axis = 1)

In [38]:
X_train = ss.fit_transform(X_train)

In [39]:
params_log = {'C': [0.01,10, 100]}

start_time = datetime.datetime.now()
gr = GridSearchCV(lr, params_log, scoring = 'roc_auc', cv = fk)
gr.fit(X_train, y_train)

print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:00:35.560601


In [40]:
print(gr.best_params_)
print(gr.best_score_)

{'C': 0.01}
0.7163321445137566


Качество почти не изменилось: 0.7164. Это связано с тем, что категориальные признаки очень слабо учитывались нашей логистической регрессией.

In [ ]:
X_train.describe()

112 героев

In [5]:
import numpy as np

X_pick = np.zeros((X_train.shape[0], 112))

for i, match_id in enumerate(X_train.index):
    for p in range(5):
        X_pick[i, X_train.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_train.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [6]:
X_pick = pd.DataFrame(X_pick)
X_train = pd.concat([X_train, X_pick], axis = 1)


In [7]:
X_train = ss.fit_transform(X_train)

NameError: name 'ss' is not defined

In [46]:
params_log = {'C': [0.01,10, 100]}

start_time = datetime.datetime.now()
gr = GridSearchCV(lr, params_log, scoring = 'roc_auc', cv = fk)
gr.fit(X_train, y_train)

print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:01:06.533746


In [47]:
print(gr.best_params_)
print(gr.best_score_)

{'C': 0.01}
0.7519624099968101


При добавлении "Мешка слов" качество заметно улучшилось и стало равно 0.7522. Это связано с тем, что идентификатор персонажа стал гораздо больше учитываться при ответе классификатора.

In [20]:
import pandas as pd

X_test = pd.read_csv("features_test.csv")
X_test = X_test.fillna(0)

import numpy as np

X_pick = np.zeros((X_test.shape[0], 112))

for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
    
X_pick = pd.DataFrame(X_pick)
X_test = pd.concat([X_test, X_pick], axis = 1)
        
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_test = ss.fit_transform(X_test)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C = 0.01)
lr.fit(X_train, y_train)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [22]:
lr.predict_proba(X_test).max()

0.996406344440937

In [23]:
lr.predict_proba(X_test).min()

0.0035936555590629915

Лучшим алгоритмом стала логистическая регрессия. Наибольшее вероятность отнесения объекта к классу на тестовой выборке равна 0.996, наименьшая  0.004